**Mapping a London Epidemic in 1854**

* This map of London was created by John Snow in 1854. London was experiencing a deadly cholera epidemic, when Snow tracked the cases on this map. The cholera cases are highlighted in black. Using this map, Snow and other scientists were able to trace the cholera outbreak to a single infected water pump.

* Dr. John Snow (1813-1858) was a famous British physician and is widely recognized as a legendary figure in the history of public health and a leading pioneer in the development of anesthesia.


* In 1854, Dr. John Snow used spatial analysis by mapping patterns and occurrences of cholera outbreaks in Soho, London. He mapped the deaths in the neighbourhood and determined that a vast majority occurred around one particular water well and that those that died used that well.



**Cholera attacks**

In [2]:
# Loading in the pandas module
import pandas as pd



In [3]:
deaths = pd.read_csv('https://raw.githubusercontent.com/FazlyRabbiBD/GIS_Remote_Sensing/main/Data_JhonShow.csv')
deaths.head()

,Number of deaths,X coordinate,Y coordinate
0,3,51.513418,-0.137930
1,2,51.513361,-0.137883
2,1,51.513317,-0.137853
3,1,51.513262,-0.137812
4,4,51.513204,-0.137767


In [4]:
deaths.tail(10)

,Number of deaths,X coordinate,Y coordinate
248,1,51.511998,-0.138123
249,1,51.511856,-0.137762
250,-999,51.513341,-0.136668
251,-999,51.513876,-0.139586
252,-999,51.514906,-0.139671
253,-999,51.512354,-0.131630
254,-999,51.512139,-0.133594
255,-999,51.511542,-0.135919
256,-999,51.510019,-0.133962
257,-999,51.511295,-0.138199


In [5]:
deaths.drop([250,251,252,253,254,255,256,257],inplace=True)

In [6]:
deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Number of deaths  250 non-null    int64  
 1   X coordinate      250 non-null    float64
 2   Y coordinate      250 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 6.0 KB


In [7]:
# Create `locations` by subsetting only Latitude and Longitude from the dataset
locations = deaths[['X coordinate', 'Y coordinate']]

# Create `deaths_list` by transforming the DataFrame to list of lists
deaths_list = locations.values.tolist()

# Check the length of the list
len(deaths_list)

250

In [8]:
# Plot the data on map (map location is provided) using folium and for loop for plotting all the points
import folium

map = folium.Map(location=[51.5132119,-0.13666], tiles='Stamen Toner', zoom_start=17)
for point in range(0, len(deaths_list)):
    folium.CircleMarker(deaths_list[point], radius=8, color='red', fill=True, fill_color='red', opacity = 0.7).add_to(map)
map

In [9]:
deaths.head()

,Number of deaths,X coordinate,Y coordinate
0,3,51.513418,-0.137930
1,2,51.513361,-0.137883
2,1,51.513317,-0.137853
3,1,51.513262,-0.137812
4,4,51.513204,-0.137767


**HeatMap**

In [10]:
from folium.plugins import HeatMap, HeatMapWithTime

In [11]:
hm = folium.Map(location=[51.5132119,-0.13666],tiles='Stamen Toner', zoom_start=17)
HeatMap(deaths_list,
        min_opacity=0.4,
        blur = 10
               ).add_to(folium.FeatureGroup(name='Heat Map').add_to(hm))
folium.LayerControl().add_to(hm)
hm

In [12]:
import numpy as np
from datetime import datetime, timedelta

**Infected Pump**

In [13]:
pumps = pd.read_csv('https://raw.githubusercontent.com/FazlyRabbiBD/GIS_Remote_Sensing/main/Data_Pumps_Jhon_Show.csv')

In [14]:
pumps.head()

,Pump Name,X coordinate,Y coordinate
0,Broad St.,51.513341,-0.136668
1,Crown Chapel,51.513876,-0.139586
2,Gt Marlborough,51.514906,-0.139671
3,Dean St.,51.512354,-0.131630
4,So Soho,51.512139,-0.133594


In [15]:
locations_pumps = pumps[['X coordinate', 'Y coordinate']]
pumps_list = locations_pumps.values.tolist()

map1 = hm
for point in range(0, len(pumps_list)):
    folium.Marker(pumps_list[point], popup=pumps['Pump Name'][point]).add_to(map1)
map1

**Data to Action**

In [16]:
deathsDf = pd.read_csv('https://raw.githubusercontent.com/FazlyRabbiBD/GIS_Remote_Sensing/main/Data_Deaths_JhonShow.csv')

In [17]:
import seaborn as sns
import matplotlib as plt

In [18]:
import plotly.express as px

In [19]:
fig = px.line(deathsDf, x="date",  y=['attacks', 'deaths'], markers=True,
              title = "Epidemic Ended by Identifing Polluted Water Pump <br><sup>Number of Cholera Deaths/Attacks before and after 8th of September 1854 (removing the pump handle)</sup>",

              )
######## renaming legends
newnames = {'attacks':'No. of Infected', 'deaths': 'No. of Deaths'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
####### line
fig.add_vline(x='1854-09-08', line_width=3, line_dash="dash", line_color="green")

###### Rectange & Text
fig.add_vrect(x0='1854-09-08', x1='1854-10-02', fillcolor="green", opacity=0.2,
              annotation_text="Incidents of Cholera Deaths/Attacks Reduced<br> after Shutdown the Polluted Water Pump at Broad St. <br> on 18 Septmber 1985<br>",
              annotation_position="inside top right",
              annotation_font_size=15,
              annotation_font_color="black"
          )

######### Footnote
fig.add_annotation(
    text = (f"Data Repository: https://github.com/FazlyRabbiBD/GIS_Remote_Sensing")
    , showarrow=False
    , x = 0
    , y = -0.15
    , xref='paper'
    , yref='paper'
    , xanchor='left'
    , yanchor='bottom'
    , xshift=-1
    , yshift=-5
    , font=dict(size=12, color="black")
    , align="left"
    ,)

fig.show()

**Animated HeatMap**

In [20]:
deathsDf[["X coordinate","Y coordinate"]]=deaths[["X coordinate","Y coordinate"]]

In [21]:
deathsDf.head(3)

,order,date,attacks,deaths,X coordinate,Y coordinate
0,1,1854-08-19,1,1,51.513418,-0.137930
1,2,1854-08-20,1,0,51.513361,-0.137883
2,3,1854-08-21,1,2,51.513317,-0.137853


In [22]:
deathsDf.set_index('date')

,order,attacks,deaths,X coordinate,Y coordinate
date,,,,,
1854-08-19,1,1,1,51.513418,-0.137930
1854-08-20,2,1,0,51.513361,-0.137883
1854-08-21,3,1,2,51.513317,-0.137853
1854-08-22,4,0,0,51.513262,-0.137812
1854-08-23,5,1,0,51.513204,-0.137767
1854-08-24,6,1,2,51.513184,-0.137537
1854-08-25,7,0,0,51.513359,-0.138200
1854-08-26,8,1,0,51.513328,-0.138045
1854-08-27,9,1,1,51.513323,-0.138276
